# filtering chartevent itemID
This one we check the same way as inputevents.
But for this table we need to also check that the value given is a numeric value, because this table does not enforce the field to be numeric.

In [1]:
# DB connection:
import psycopg2
import datetime
import sys
from operator import itemgetter, attrgetter, methodcaller
import numpy as np
import itertools
import os.path
import matplotlib.pyplot as plt
import math
%matplotlib inline

# connect DB

In [2]:
try:
    conn = psycopg2.connect("dbname='mimic' user='mimic' host='localhost' password='meladymimic315'")
except:
    print "I am unable to connect to the database"
print 'connection established'

connection established


# load admissionID and itemID

In [3]:
_adm = np.load('res/admission_ids.npy').tolist()
admission_ids = _adm['admission_ids']
admission_ids_txt = _adm['admission_ids_txt']

db = np.load('res/itemids.npy').tolist()
input_itemid = db['input']
output_itemid = db['output']
chart_itemid = db['chart']
lab_itemid = db['lab']
microbio_itemid = db['microbio']
prescript_itemid = db['prescript']

For each itemid first we check the unit of measurement as we did in inputevents. Then we continue to check if the observations more than 20% is not numeric then we also discard that itemID.

In [6]:
valid_chart = []
valid_chart_unit = []
valid_chart_cate = []
dropped = []

for i in chart_itemid:
    # for each itemID select number of rows group by unit of measurement.
    cur = conn.cursor()
    cur.execute('SELECT valueuom, count(valueuom) FROM mimiciii.chartevents WHERE itemid = '+ str(i) +' and hadm_id in ('+admission_ids_txt+') group by valueuom')
    chartunits = cur.fetchall()
    chartunits = sorted(chartunits, key=lambda tup: tup[1])
    chartunits.reverse()

    # calculate percentage of the top frequent unit compared to all observation.
    total = 0
    for c in chartunits:
        total += c[1]
    if(total == 0 ): continue;
    percentage = float(chartunits[0][1]) / total * 100.
    
    # select label of the itemID
    cur = conn.cursor()
    cur.execute('SELECT label FROM mimiciii.d_items WHERE itemid = '+ str(i))
    label = cur.fetchone()
    
    # print and discard the one with many different units manually.
#     print label[0]
    print "CHART "+str(i) + "\t" + "{:.2f}".format(percentage) +'\t'+ str(chartunits)
    
    # count number of observation that has non numeric value
    cur = conn.cursor()
    cur.execute('SELECT count(*) FROM mimiciii.chartevents WHERE itemid = '+ str(i) +' and hadm_id in ('+admission_ids_txt+') and valuenum is null')
    notnum = cur.fetchone()
    
    # total number of observation
    cur = conn.cursor()
    cur.execute('SELECT count(*) FROM mimiciii.chartevents WHERE itemid = '+ str(i) +' and hadm_id in ('+admission_ids_txt+')')
    total = cur.fetchone()
    
    # if the percentage of numeric number is less, then dropped it, and make it categorical feature.
    percentage =float(total[0] -notnum[0])*100 / total[0]
    print "Numeric observation :" + "{:.4f}%".format(percentage)+ " ( NOTNUM= " + str(notnum[0]) + " / ALL= " + str(total[0]) + " ) "
    if(percentage < 80): 
        print 'dropped\n'
        dropped.append(i)
        continue;
    print ''
    
    valid_chart.append(i);
    valid_chart_unit.append(chartunits[0][0]);

CHART 227801	100.00	[('', 7071L)]
Numeric observation :100.0000% ( NOTNUM= 0 / ALL= 7071 ) 

CHART 227063	100.00	[('', 7L)]
Numeric observation :100.0000% ( NOTNUM= 0 / ALL= 7 ) 

CHART 228369	100.00	[('L/min', 715L)]
Numeric observation :100.0000% ( NOTNUM= 0 / ALL= 715 ) 

CHART 227419	100.00	[('', 456L)]
Numeric observation :100.0000% ( NOTNUM= 0 / ALL= 456 ) 

CHART 226185	100.00	[('', 5664L)]
Numeric observation :100.0000% ( NOTNUM= 0 / ALL= 5664 ) 

CHART 225771	100.00	[('', 519L)]
Numeric observation :100.0000% ( NOTNUM= 0 / ALL= 519 ) 

CHART 224663	100.00	[('', 18966L)]
Numeric observation :100.0000% ( NOTNUM= 0 / ALL= 18966 ) 

CHART 220603	100.00	[('mg/dL', 1402L)]
Numeric observation :100.0000% ( NOTNUM= 0 / ALL= 1402 ) 

CHART 226134	100.00	[('', 2801L)]
Numeric observation :100.0000% ( NOTNUM= 0 / ALL= 2801 ) 

CHART 227007	100.00	[('', 6L)]
Numeric observation :100.0000% ( NOTNUM= 0 / ALL= 6 ) 

CHART 226092	100.00	[('mmHg', 165L)]
Numeric observation :100.0000% ( NOTNUM

In [30]:
np.save('res/filtered_chart',{'id':valid_chart,'unit':valid_chart_unit});
np.save('res/filtered_chart_cate',{'id':[223758],'unit':None});
print "saved !";

saved !


In [10]:
for i in admission_ids:
    print str(i)+',',

185131, 164444, 191210, 197882, 140212, 149609, 113210, 144497, 194406, 131870, 170691, 197979, 102738, 157477, 156416, 122781, 137515, 135053, 130978, 170590, 166642, 101853, 162296, 128584, 196485, 187989, 120113, 193878, 186894, 107019, 175538, 101424, 125757, 131243, 186618, 112235, 178138, 197042, 195762, 180913, 151200, 158793, 107667, 192742, 198726, 100050, 160506, 136865, 145127, 119375, 194032, 103674, 137417, 138450, 181893, 115231, 170760, 101095, 106042, 138410, 150877, 133430, 100347, 182358, 105357, 174483, 136336, 199640, 122850, 145303, 148782, 138046, 169775, 190574, 168925, 160723, 199369, 113295, 135215, 106361, 157358, 148117, 194640, 188690, 180926, 150223, 157261, 155343, 198959, 167095, 154347, 157942, 171641, 123500, 194096, 177915, 125781, 137232, 150895, 132449, 113380, 100185, 120478, 121778, 154490, 143175, 150959, 186013, 180170, 179324, 182054, 184266, 146466, 170474, 143994, 190586, 176653, 166622, 137969, 184442, 184495, 173080, 131337, 146010, 147277, 